In [ ]:
import pymysql
import pprint
import pandas as pd
from matplotlib import pyplot as plt
import passwords

mysqlip = passwords.mysqlip
mysqlpassword = passwords.mysqlpassword
mysqlusername = passwords.mysqlusername
mysqldb = passwords.mysqldb_expungement

pp = pprint.PrettyPrinter(indent=4)

In [ ]:
def sendSQLCommand(sql):
    try:
        connection = pymysql.connect(host=mysqlip,
                                     user=mysqlusername,
                                     password=mysqlpassword,
                                     db=mysqldb,
                                     charset='utf8mb4',
                                     cursorclass=pymysql.cursors.DictCursor)

        cursor = connection.cursor()
        cursor.execute(sql)
        dockets = cursor.fetchall()
        cursor.close()
        return dockets
    except Exception as e:
        print(e)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        return None

In [ ]:
#Outcome Breakdown Joco

#Get Eligible First
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory NOT IN ('Ineligible','Unknown Crime Category')
AND WaitTime IS NULL
AND FelonyConvictionLastTwoYears = False
AND FelonyPending = False
AND RegistryInRecord IS NULL
AND County = 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
joco_eligible = results[0]['COUNT(*)']

#Get Not Yet Eligible
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory NOT IN ('Ineligible','Unknown Crime Category')
AND WaitTime IS NOT NULL
AND FelonyConvictionLastTwoYears = False
AND FelonyPending = False
AND RegistryInRecord IS NULL
AND County = 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
joco_not_yet_eligible = results[0]['COUNT(*)']

#Never Eligible
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory = 'Ineligible'
or RegistryInRecord > '2100-01-01'
AND County = 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
joco_never_eligible = results[0]['COUNT(*)']

#Unknown Eligible
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory = 'Unknown Crime Category'
AND County = 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
joco_unknown = results[0]['COUNT(*)']

newresults = [
    {'Eligibility':'Eligible','Convictions':joco_eligible},
    {'Eligibility':'Not Yet Eligible','Convictions':joco_not_yet_eligible},
    {'Eligibility':'Ineligible','Convictions':joco_never_eligible},
    {'Eligibility':'Unknown','Convictions':joco_unknown}
]
df = pd.DataFrame(newresults)
display(df)
title = 'Conviction Expungement Eligibilty in Johnson County'
temp = df.plot(kind="bar",x='Eligibility',ylabel="Amount of Convictions",title=title)
temp.figure.savefig("Graphs/" + title + ".png",bbox_inches="tight")


In [ ]:
#Non Johnson County Expungement Eligibility

#Get Eligible First Joco
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory NOT IN ('Ineligible','Unknown Crime Category')
AND WaitTime IS NULL
AND FelonyConvictionLastTwoYears = False
AND FelonyPending = False
AND RegistryInRecord IS NULL
AND County = 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
joco_eligible = results[0]['COUNT(*)']

#Get Eligible First State
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory NOT IN ('Ineligible','Unknown Crime Category')
AND WaitTime IS NULL
AND FelonyConvictionLastTwoYears = False
AND FelonyPending = False
AND RegistryInRecord IS NULL
AND County != 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
state_eligible = results[0]['COUNT(*)']

#Get Not Yet Eligible
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory NOT IN ('Ineligible','Unknown Crime Category')
AND WaitTime IS NOT NULL
AND FelonyConvictionLastTwoYears = False
AND FelonyPending = False
AND RegistryInRecord IS NULL
AND County = 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
joco_not_yet_eligible = results[0]['COUNT(*)']

#Get not yet eligible state
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory NOT IN ('Ineligible','Unknown Crime Category')
AND WaitTime IS NOT NULL
AND FelonyConvictionLastTwoYears = False
AND FelonyPending = False
AND RegistryInRecord IS NULL
AND County != 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
state_not_yet_eligible = results[0]['COUNT(*)']

#Never Eligible
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory = 'Ineligible'
or RegistryInRecord > '2100-01-01'
AND County = 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
joco_never_eligible = results[0]['COUNT(*)']

#State Never Eligible
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory = 'Ineligible'
or RegistryInRecord > '2100-01-01'
AND County = 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
state_never_eligible = results[0]['COUNT(*)']

#Unknown
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory = 'Unknown Crime Category'
AND County = 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
joco_unknown = results[0]['COUNT(*)']

#Unknown State 
sql = '''
SELECT COUNT(*) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory = 'Unknown Crime Category'
AND County = 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
state_unknown = results[0]['COUNT(*)']

total_state_eligible_convictions = state_not_yet_eligible + state_eligible
joco_eligible_percent = joco_eligible * 100/(joco_eligible + joco_not_yet_eligible)
joco_not_yet_eligible_percent = joco_not_yet_eligible * 100/(joco_eligible + joco_not_yet_eligible)

# joco_eligible_percent = joco_eligible * 100/joco_total_eligible_not_yet_eligible
# joco_not_yet_eligible_percent = joco_not_yet_eligible * 100/joco_total_eligible_not_yet_eligible

state_results =[
    {'Eligibility':'Eligible','Original Convictions':state_eligible,'Adjusted Proportional Convictions':joco_eligible_percent * total_state_eligible_convictions/100},
    {'Eligibility':'Not Yet Eligible','Original Convictions':state_not_yet_eligible,'Adjusted Proportional Convictions':joco_not_yet_eligible_percent * total_state_eligible_convictions/100},
    {'Eligibility':'Ineligible','Original Convictions':state_not_yet_eligible},
    {'Eligibility':'Unknown','Original Convictions':state_unknown},
]

df = pd.DataFrame(state_results)
display(df)
title = 'Conviction Expungement Eligibilty in Non-Johnson County'
temp = df.plot(kind="bar",x='Eligibility',ylabel="Amount of Convictions",title=title)
temp.figure.savefig("Graphs/" + title + ".png",bbox_inches="tight")

In [ ]:
#Number of Cases Where All Charges Are Eligible 2022
final = []

sql = '''
SELECT COUNT(*) as Eligible FROM (SELECT CaseNumber, sum(CASE WHEN ChargeCategory IN ('Ineligible','Unknown Crime Category')
or WaitTime IS NOT NULL
or FelonyConvictionLastTwoYears = True
or FelonyPending = True
or RegistryInRecord IS NOT NULL
THEN 1 ELSE 0 END) 
as case_exp FROM kansas_expungement2022.analysis2022 WHERE County = 'Johnson County' GROUP BY casenumber) as tmp WHERE tmp.case_exp =0
'''
temp = {}
results = sendSQLCommand(sql)
print(results)
final.append({'County':'Johnson County','Cases':results[0]['Eligible']})
#temp['Johnson County'] = results[0]['Eligible']

sql = '''
SELECT COUNT(*) as Eligible FROM (SELECT CaseNumber, sum(CASE WHEN ChargeCategory IN ('Ineligible','Unknown Crime Category')
or WaitTime IS NOT NULL
or FelonyConvictionLastTwoYears = True
or FelonyPending = True
or RegistryInRecord IS NOT NULL
THEN 1 ELSE 0 END) 
as case_exp FROM kansas_expungement2022.analysis2022 WHERE County != 'Johnson County' GROUP BY casenumber) as tmp WHERE tmp.case_exp =0
'''
results = sendSQLCommand(sql)
print(results)
#temp['Not Johnson County'] = results[0]['Eligible']
final.append({'County':'Non-Johnson County','Cases':results[0]['Eligible']})

df = pd.DataFrame(final)
display(df)
title = 'Criminal Cases Where All Convictions \n are Eligible for Expungement'
temp = df.plot(kind="bar",title=title,ylabel='Amount of Cases',x = 'County')
temp.figure.savefig("Graphs/" + title.replace("\n","") + ".png",bbox_inches="tight")

In [ ]:
#Number of People Where all of the charges are eligible 2022
final = []

sql = '''
SELECT COUNT(DISTINCT Person) FROM
kansas_expungement2022.analysis2022
WHERE Person NOT IN (SELECT DISTINCT(Person) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory IN ('Ineligible','Unknown Crime Category')
or WaitTime IS NOT NULL
or FelonyConvictionLastTwoYears = True
or FelonyPending = True
or RegistryInRecord IS NOT NULL
)
AND County = 'Johnson County'
'''
temp = {}
results = sendSQLCommand(sql)
print(results)
final.append({'County':'Johnson County','People':results[0]['COUNT(DISTINCT Person)']})

#temp['Johnson County'] = results[0]['COUNT(DISTINCT Person)']

sql = '''
SELECT COUNT(DISTINCT Person) FROM
kansas_expungement2022.analysis2022
WHERE Person NOT IN (SELECT DISTINCT(Person) FROM kansas_expungement2022.analysis2022 WHERE 
ChargeCategory IN ('Ineligible','Unknown Crime Category')
or WaitTime IS NOT NULL
or FelonyConvictionLastTwoYears = True
or FelonyPending = True
or RegistryInRecord IS NOT NULL
)
AND County != 'Johnson County'
'''
results = sendSQLCommand(sql)
print(results)
#temp['Not Johnson County'] = results[0]['COUNT(DISTINCT Person)']
final.append({'County':'Non-Johnson County','People':results[0]['COUNT(DISTINCT Person)']})


df = pd.DataFrame(final)
display(df)
title = 'People With Fully Eligible Records'
temp = df.plot(kind="bar",title=title,x='County',xlabel='County',ylabel='Amount of People')
temp.figure.savefig("Graphs/" + title + ".png",bbox_inches="tight")

In [ ]:
#Breakdown by Expungement Category Non-Johnson County 2022
sql = '''
SELECT ExpungementCategory,COUNT(*) as Convictions FROM kansas_expungement2022.analysis2022 GROUP BY ExpungementCategory ORDER BY Convictions DESC
'''
results = sendSQLCommand(sql)
newresults = []
for item in results:
    if(item['ExpungementCategory'] == 'Error'):
        item['ExpungementCategory'] = 'Unknown'
    elif(item['ExpungementCategory'] == 'Drug 4'):
        item['ExpungementCategory'] = 'Drug Felony Level 4'
    elif(item['ExpungementCategory'] == 'Non Drug 678910'):
        item['ExpungementCategory'] = 'Non Drug Felony Level 6-10'
    elif(item['ExpungementCategory'] == 'Drug 5'):
        item['ExpungementCategory'] = 'Drug Felony Level 5'
    elif(item['ExpungementCategory'] == 'Drug 123'):
        item['ExpungementCategory'] = 'Drug Felony Level 1-3'
    elif(item['ExpungementCategory'] == 'Non Drug 12345'):
        item['ExpungementCategory'] = 'Non Drug Felony Level 1-5'
    elif(item['ExpungementCategory'] == 'Drug 1234'):
        item['ExpungementCategory'] = 'Drug Felony Level 1-4'
    elif(item['ExpungementCategory'] == 'Drug ABC'):
        item['ExpungementCategory'] = 'Drug Felony Level A-C'
    newresults.append(item)
df = pd.DataFrame(newresults)
#df = df.sort_values(by=['Convictions'],ascending=False)
display(df)
title = 'Breakdown of Convictions By Charge Category'
temp = df.plot(kind="bar",x='ExpungementCategory',title=title ,xlabel = 'Charge Category',ylabel='Convictions')
temp.figure.savefig("Graphs/" + title + ".png",bbox_inches="tight")

In [ ]:
#Breakdown by Waiting Period Category
sql = "SELECT ChargeCategory,COUNT(*) as Convictions FROM  kansas_expungement2022.analysis2022 GROUP BY ChargeCategory ORDER BY COUNT(*)  DESC"
results = sendSQLCommand(sql)
newresults = []
print(results)
for item in results:
    if(item['ChargeCategory'] == 'Unknown Crime Category'):
        item['ChargeCategory'] = 'Unknown'
    newresults.append(item)
df = pd.DataFrame(newresults)
display(df)
title = 'Breakdown of Convictions By Waiting Period Category'
temp = df.head(5).plot(kind="bar",x='ChargeCategory',title=title,ylabel='Convictions',xlabel = 'Waiting Period Category')
temp.figure.savefig("Graphs/" + title + ".png",bbox_inches="tight")

In [ ]:
#Convictions with wait times
sql = '''SELECT YEAR(WaitTime),COUNT(*) as Convictions FROM kansas_expungement2022.analysis2022 
WHERE WaitTime != 0 and WaitTime != 1 AND County = 'Johnson County' 
GROUP BY YEAR(WaitTime) ORDER BY YEAR(WaitTime) ASC'''
results = sendSQLCommand(sql)
df = pd.DataFrame(results).head(10)
display(df)
title = 'Johnson County Convictions Eligible In the Future'
temp = df.plot(kind="bar",x='YEAR(WaitTime)',title=title ,xlabel = 'Year of Eligibilty',ylabel='Convictions')
temp.figure.savefig("Graphs/" + title +".png",bbox_inches="tight")

sql = '''SELECT YEAR(WaitTime),COUNT(*) as Convictions FROM kansas_expungement2022.analysis2022 
WHERE WaitTime != 0 and WaitTime != 1 AND County != 'Johnson County' 
GROUP BY YEAR(WaitTime) ORDER BY YEAR(WaitTime) ASC'''
results = sendSQLCommand(sql)
df = pd.DataFrame(results).head(10)
display(df)

title = 'Non-Johnson County Convictions \nEligible In the Future'
temp = df.plot(kind="bar",x='YEAR(WaitTime)',title=title,xlabel = 'Year of Eligibilty',ylabel='Convictions')
temp.figure.savefig("Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Number of Cases prevented by felonies. 2022
combined = []
final = []
sql = '''SELECT SUM(FelonyConvictionLastTwoYears),SUM(FelonyPending) FROM kansas_expungement2022.analysis2022 
WHERE WaitTime IS NULL 
AND Outcome = 'Not Yet Eligible'
AND (FelonyConvictionLastTwoYears = True or FelonyPending = True)
AND County = 'Johnson County'
AND RegistryInRecord IS NULL
'''
results = sendSQLCommand(sql)
final.append({'Eligibility Block':'Felony Within Two Years','Convictions':results[0]['SUM(FelonyConvictionLastTwoYears)']})
final.append({'Eligibility Block':'Felony Pending','Convictions':results[0]['SUM(FelonyPending)']})

combined.append({'Eligibility Block':'Felony Within Two Years','Johnson County':results[0]['SUM(FelonyConvictionLastTwoYears)']})
combined.append({'Eligibility Block':'Felony Pending','Johnson County':results[0]['SUM(FelonyPending)']})

df = pd.DataFrame(final)
display(df)
temp = df.plot(kind="bar",title='Johnson County Convictions Blocked by Felony',x='Eligibility Block',ylabel='Convictions')
#temp.figure.savefig("Johnson County Convictions Blocked by Felony.png",bbox_inches="tight")

#Number of Cases prevented by felonies.
final = []
sql = '''SELECT SUM(FelonyConvictionLastTwoYears),SUM(FelonyPending) FROM kansas_expungement2022.analysis2022  
WHERE  WaitTime IS NULL 
AND Outcome = 'Not Yet Eligible'
AND (FelonyConvictionLastTwoYears = True or FelonyPending = True)
AND County != 'Johnson County'
AND RegistryInRecord IS NULL
'''
results = sendSQLCommand(sql)
final.append({'Eligibility Block':'Felony Within Two Years','Convictions':results[0]['SUM(FelonyConvictionLastTwoYears)']})
final.append({'Eligibility Block':'Felony Pending','Convictions':results[0]['SUM(FelonyPending)']})

combined[0]['Non-Johnson County'] = results[0]['SUM(FelonyConvictionLastTwoYears)']
combined[1]['Non-Johnson County'] = results[0]['SUM(FelonyPending)'] 
                    
df = pd.DataFrame(final)
display(df)
temp = df.plot(kind="bar",title='Non Johnson County Convictions Blocked by Past Felony',x='Eligibility Block',ylabel='Convictions')
#temp.figure.savefig("Non Johnson County Convictions Blocked by Felony.png",bbox_inches="tight")

df = pd.DataFrame(combined)
title = 'Convictions Blocked By A Felony Conviction \nWithin Two Years or a Pending Felony Charge'
temp = df.plot(kind="bar",x='Eligibility Block',ylabel="Amount of Convictions",title=title)
temp.figure.savefig("Graphs/" + title.replace("\n","") + ".png",bbox_inches="tight")
